#  Network Security
## Background
Refer to [OSI Model](../computer-networks/osi_model.ipynb#osi_model)

## Security Weakness
The origin design of Internet did not take security into account.

Thus, below are the possible types of attacks that can be performed (derived from the [CIA Triad](./introduction.ipynb#cia-triad)):
* Interception: Unauthorized viewing
* Modification: Unauthorized change
* Fabrication: Unauthorized creation
* Interruption: Preventing of authorized access

At any layer of the OSI model, the attacker can modify **both the header and the data**.

## Name Resolution
Each entity has a name, which is different for each layer of the OSI model.

For instance: 
* at layer 7 (Application level), the entity is denoted by their url: "www.google.com"
* at layer 3 (Network level), the entity is denoted by their ip address: 74.125.200.99
* at layer 2 (Datalink level), the entity is denoted by their MAC address: 59:8c:c8:f3:8d:94

Thus, as information travel down the layers, they need a way of translating higher level addresses into lower level ones, through a **resolution protocol**.

During the design of resolution protocols, security was not accounted for, thus it is easy for attackers to influence the outcome of name resolutions.

With the background knowledge about [ARP](../computer-networks/osi_model.ipynb#arp), we will discuss possible vulnerabilities.


### Authentication in DNS
Each DNS query contains a 16-bit number known as the **Query ID (QID)**.
The response from the server also contains the same QID.
The requester only accepts the response if the QID matches.

Note that QID was not designed primarily for authentication, but rather as a way to find the DNS response that the requester is interested in.

Also, further note that there is no encryption or MAC involved in the protocol, thus **neither confidentiality nor integrity is guaranteed**.

### DNS Spoofing
The attacker can perform the following:
1. Victim makes a DNS query for a certain domain to find the IP address
2. Attacker see this query, and quickly replies with the IP address of his malicious server together with the corresponding QID
3. The DNS server sees the victim's query, and sends the DNS response
3. Because the attacker responded faster than the legitimate DNS server, the victim sees the malicious query first, and thus visits the malicious server instead of the genuine server

## Denial of Service (DoS)
DoS targets the availability of a service.

Types of DoS attacks:

|   | Terminating service | Exhausting resources |
|---| ---- | ---- |
|.  |.    |.     |